### Import Dependencies and Setup

In [ ]:
# Import required libraries
import os
import asyncio
import logging
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import time

# Interactive Brokers imports
from ib_insync import IB, Forex, MarketOrder, LimitOrder, Ticker
from ib_insync import util

log_dir = "logs"
log_file = "forex_trading_bot.log"
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, log_file)

# Create log file if it doesn't exist
if not os.path.exists(log_path):
    open(log_path, "a").close()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(log_path)
    ]
)

# Suppress ib_insync debug logs
logging.getLogger('ib_insync').setLevel(logging.WARNING)

print("✅ Dependencies imported successfully")
print("✅ Logging configured")

### IB Connection

In [ ]:
class LiveForexConnection:
    """Manages Interactive Brokers live connection for forex trading"""
    
    def __init__(self):
        """Initialise IB connection for live trading environment"""
        self.ib = IB()
        self.logger = logging.getLogger('LiveForexConnection')
        
        # Live trading connection parameters
        self.host = '127.0.0.1'
        self.port = 7496  # Live trading port
        self.client_id = 1
        self.connected = False
        
        # Configure IB for Jupyter environment
        util.startLoop()
    
    def connect(self) -> bool:
        """
        Establish connection to IB TWS/Gateway for live trading
        
        Returns:
            bool: True if connection successful, False otherwise
        """
        try:
            if self.ib.isConnected():
                self.logger.info("Already connected to live environment")
                self.connected = True
                return True
            
            # Use connect without blocking for Jupyter
            self.ib.connect(self.host, self.port, clientId=self.client_id)
            
            if self.ib.isConnected():
                self.connected = True
                self.logger.info("Successfully connected to live trading environment")
                print("✅ Connected to Interactive Brokers live environment")
                return True
            else:
                self.connected = False
                self.logger.error("Failed to connect to live environment")
                print("❌ Failed to connect to live environment")
                return False
                
        except Exception as e:
            self.connected = False
            self.logger.error(f"Connection error: {e}")
            print(f"❌ Connection error: {e}")
            return False
    
    def disconnect(self):
        """Disconnect from IB"""
        try:
            if self.ib.isConnected():
                self.ib.disconnect()
                self.connected = False
                self.logger.info("Disconnected from live environment")
                print("✅ Disconnected from live environment")
        except Exception as e:
            self.logger.error(f"Disconnect error: {e}")
    
    def is_connected(self) -> bool:
        """Check if connection is active"""
        return self.ib.isConnected() and self.connected
    
    def get_account_info(self) -> Dict:
        """Get account information"""
        if not self.is_connected():
            return {}
        
        try:
            accounts = self.ib.managedAccounts()
            return {
                'accounts': accounts,
                'environment': 'live',
                'connected': True
            }
        except Exception as e:
            self.logger.error(f"Error getting account info: {e}")
            return {}

# Alternative connection approach if the above still fails
class LiveForexConnectionAlternative:
    """Alternative IB connection approach for problematic environments"""
    
    def __init__(self):
        """Initialise IB connection for live trading environment"""
        self.logger = logging.getLogger('LiveForexConnectionAlt')
        
        # Live trading connection parameters
        self.host = '127.0.0.1'
        self.port = 7496
        self.client_id = 2  # Different client ID
        self.connected = False
        self.ib = None
    
    def connect(self) -> bool:
        """
        Establish connection using nest_asyncio approach
        
        Returns:
            bool: True if connection successful, False otherwise
        """
        try:
            # Import nest_asyncio for Jupyter compatibility
            import nest_asyncio
            nest_asyncio.apply()
            
            # Create new IB instance
            self.ib = IB()
            
            # Connect with timeout
            self.ib.connect(self.host, self.port, clientId=self.client_id, timeout=10)
            
            if self.ib.isConnected():
                self.connected = True
                self.logger.info("Successfully connected to live trading environment")
                print("✅ Connected to Interactive Brokers live environment (Alternative method)")
                return True
            else:
                self.connected = False
                self.logger.error("Failed to connect to live environment")
                print("❌ Failed to connect to live environment")
                return False
                
        except ImportError:
            print("❌ nest_asyncio not available. Install with: pip install nest-asyncio")
            return False
        except Exception as e:
            self.connected = False
            self.logger.error(f"Connection error: {e}")
            print(f"❌ Connection error: {e}")
            return False
    
    def disconnect(self):
        """Disconnect from IB"""
        try:
            if self.ib and self.ib.isConnected():
                self.ib.disconnect()
                self.connected = False
                self.logger.info("Disconnected from live environment")
                print("✅ Disconnected from live environment")
        except Exception as e:
            self.logger.error(f"Disconnect error: {e}")
    
    def is_connected(self) -> bool:
        """Check if connection is active"""
        return self.ib and self.ib.isConnected() and self.connected
    
    def get_account_info(self) -> Dict:
        """Get account information"""
        if not self.is_connected():
            return {}
        
        try:
            accounts = self.ib.managedAccounts()
            return {
                'accounts': accounts,
                'environment': 'live',
                'connected': True
            }
        except Exception as e:
            self.logger.error(f"Error getting account info: {e}")
            return {}

print("✅ Fixed LiveForexConnection classes created")

### World Currencies Database

In [ ]:
class WorldCurrenciesDatabase:
    """Database of all world currencies from 195 countries"""
    
    def __init__(self):
        """Initialise with complete world currencies database"""
        self.logger = logging.getLogger('WorldCurrenciesDatabase')
        
        # Complete list of world currencies by country/region
        self.world_currencies = {
            # Major Developed Markets
            'USD': {'name': 'US Dollar', 'country': 'United States', 'region': 'North America', 'category': 'Major'},
            'EUR': {'name': 'Euro', 'country': 'European Union', 'region': 'Europe', 'category': 'Major'},
            'GBP': {'name': 'British Pound', 'country': 'United Kingdom', 'region': 'Europe', 'category': 'Major'},
            'JPY': {'name': 'Japanese Yen', 'country': 'Japan', 'region': 'Asia', 'category': 'Major'},
            'CHF': {'name': 'Swiss Franc', 'country': 'Switzerland', 'region': 'Europe', 'category': 'Major'},
            'CAD': {'name': 'Canadian Dollar', 'country': 'Canada', 'region': 'North America', 'category': 'Major'},
            'AUD': {'name': 'Australian Dollar', 'country': 'Australia', 'region': 'Oceania', 'category': 'Major'},
            'NZD': {'name': 'New Zealand Dollar', 'country': 'New Zealand', 'region': 'Oceania', 'category': 'Major'},
            
            # European Currencies
            'SEK': {'name': 'Swedish Krona', 'country': 'Sweden', 'region': 'Europe', 'category': 'Minor'},
            'NOK': {'name': 'Norwegian Krone', 'country': 'Norway', 'region': 'Europe', 'category': 'Minor'},
            'DKK': {'name': 'Danish Krone', 'country': 'Denmark', 'region': 'Europe', 'category': 'Minor'},
            'PLN': {'name': 'Polish Zloty', 'country': 'Poland', 'region': 'Europe', 'category': 'Minor'},
            'CZK': {'name': 'Czech Koruna', 'country': 'Czech Republic', 'region': 'Europe', 'category': 'Minor'},
            'HUF': {'name': 'Hungarian Forint', 'country': 'Hungary', 'region': 'Europe', 'category': 'Minor'},
            'RON': {'name': 'Romanian Leu', 'country': 'Romania', 'region': 'Europe', 'category': 'Exotic'},
            'BGN': {'name': 'Bulgarian Lev', 'country': 'Bulgaria', 'region': 'Europe', 'category': 'Exotic'},
            'HRK': {'name': 'Croatian Kuna', 'country': 'Croatia', 'region': 'Europe', 'category': 'Exotic'},
            'RSD': {'name': 'Serbian Dinar', 'country': 'Serbia', 'region': 'Europe', 'category': 'Exotic'},
            'ISK': {'name': 'Icelandic Krona', 'country': 'Iceland', 'region': 'Europe', 'category': 'Exotic'},
            'ALL': {'name': 'Albanian Lek', 'country': 'Albania', 'region': 'Europe', 'category': 'Exotic'},
            'MKD': {'name': 'Macedonian Denar', 'country': 'North Macedonia', 'region': 'Europe', 'category': 'Exotic'},
            'BAM': {'name': 'Bosnia-Herzegovina Convertible Mark', 'country': 'Bosnia and Herzegovina', 'region': 'Europe', 'category': 'Exotic'},
            'MDL': {'name': 'Moldovan Leu', 'country': 'Moldova', 'region': 'Europe', 'category': 'Exotic'},
            'UAH': {'name': 'Ukrainian Hryvnia', 'country': 'Ukraine', 'region': 'Europe', 'category': 'Exotic'},
            'BYN': {'name': 'Belarusian Ruble', 'country': 'Belarus', 'region': 'Europe', 'category': 'Exotic'},
            'RUB': {'name': 'Russian Ruble', 'country': 'Russia', 'region': 'Europe/Asia', 'category': 'Exotic'},
            
            # Asian Currencies
            'CNY': {'name': 'Chinese Yuan', 'country': 'China', 'region': 'Asia', 'category': 'Minor'},
            'HKD': {'name': 'Hong Kong Dollar', 'country': 'Hong Kong', 'region': 'Asia', 'category': 'Minor'},
            'SGD': {'name': 'Singapore Dollar', 'country': 'Singapore', 'region': 'Asia', 'category': 'Minor'},
            'KRW': {'name': 'South Korean Won', 'country': 'South Korea', 'region': 'Asia', 'category': 'Minor'},
            'TWD': {'name': 'Taiwan Dollar', 'country': 'Taiwan', 'region': 'Asia', 'category': 'Exotic'},
            'INR': {'name': 'Indian Rupee', 'country': 'India', 'region': 'Asia', 'category': 'Exotic'},
            'IDR': {'name': 'Indonesian Rupiah', 'country': 'Indonesia', 'region': 'Asia', 'category': 'Exotic'},
            'MYR': {'name': 'Malaysian Ringgit', 'country': 'Malaysia', 'region': 'Asia', 'category': 'Exotic'},
            'THB': {'name': 'Thai Baht', 'country': 'Thailand', 'region': 'Asia', 'category': 'Exotic'},
            'PHP': {'name': 'Philippine Peso', 'country': 'Philippines', 'region': 'Asia', 'category': 'Exotic'},
            'VND': {'name': 'Vietnamese Dong', 'country': 'Vietnam', 'region': 'Asia', 'category': 'Exotic'},
            'PKR': {'name': 'Pakistani Rupee', 'country': 'Pakistan', 'region': 'Asia', 'category': 'Exotic'},
            'BDT': {'name': 'Bangladeshi Taka', 'country': 'Bangladesh', 'region': 'Asia', 'category': 'Exotic'},
            'LKR': {'name': 'Sri Lankan Rupee', 'country': 'Sri Lanka', 'region': 'Asia', 'category': 'Exotic'},
            'NPR': {'name': 'Nepalese Rupee', 'country': 'Nepal', 'region': 'Asia', 'category': 'Exotic'},
            'BTN': {'name': 'Bhutanese Ngultrum', 'country': 'Bhutan', 'region': 'Asia', 'category': 'Exotic'},
            'MVR': {'name': 'Maldivian Rufiyaa', 'country': 'Maldives', 'region': 'Asia', 'category': 'Exotic'},
            'AFN': {'name': 'Afghan Afghani', 'country': 'Afghanistan', 'region': 'Asia', 'category': 'Exotic'},
            'KZT': {'name': 'Kazakh Tenge', 'country': 'Kazakhstan', 'region': 'Asia', 'category': 'Exotic'},
            'UZS': {'name': 'Uzbek Som', 'country': 'Uzbekistan', 'region': 'Asia', 'category': 'Exotic'},
            'KGS': {'name': 'Kyrgyz Som', 'country': 'Kyrgyzstan', 'region': 'Asia', 'category': 'Exotic'},
            'TJS': {'name': 'Tajik Somoni', 'country': 'Tajikistan', 'region': 'Asia', 'category': 'Exotic'},
            'TMT': {'name': 'Turkmen Manat', 'country': 'Turkmenistan', 'region': 'Asia', 'category': 'Exotic'},
            'MNT': {'name': 'Mongolian Tugrik', 'country': 'Mongolia', 'region': 'Asia', 'category': 'Exotic'},
            'KPW': {'name': 'North Korean Won', 'country': 'North Korea', 'region': 'Asia', 'category': 'Exotic'},
            'LAK': {'name': 'Lao Kip', 'country': 'Laos', 'region': 'Asia', 'category': 'Exotic'},
            'KHR': {'name': 'Cambodian Riel', 'country': 'Cambodia', 'region': 'Asia', 'category': 'Exotic'},
            'MMK': {'name': 'Myanmar Kyat', 'country': 'Myanmar', 'region': 'Asia', 'category': 'Exotic'},
            'BND': {'name': 'Brunei Dollar', 'country': 'Brunei', 'region': 'Asia', 'category': 'Exotic'},
            'KWD': {'name': 'Kuwaiti Dinar', 'country': 'Kuwait', 'region': 'Asia', 'category': 'Exotic'},
            'BHD': {'name': 'Bahraini Dinar', 'country': 'Bahrain', 'region': 'Asia', 'category': 'Exotic'},
            'QAR': {'name': 'Qatari Riyal', 'country': 'Qatar', 'region': 'Asia', 'category': 'Exotic'},
            'AED': {'name': 'UAE Dirham', 'country': 'United Arab Emirates', 'region': 'Asia', 'category': 'Exotic'},
            'OMR': {'name': 'Omani Rial', 'country': 'Oman', 'region': 'Asia', 'category': 'Exotic'},
            'SAR': {'name': 'Saudi Riyal', 'country': 'Saudi Arabia', 'region': 'Asia', 'category': 'Exotic'},
            'YER': {'name': 'Yemeni Rial', 'country': 'Yemen', 'region': 'Asia', 'category': 'Exotic'},
            'IQD': {'name': 'Iraqi Dinar', 'country': 'Iraq', 'region': 'Asia', 'category': 'Exotic'},
            'IRR': {'name': 'Iranian Rial', 'country': 'Iran', 'region': 'Asia', 'category': 'Exotic'},
            'JOD': {'name': 'Jordanian Dinar', 'country': 'Jordan', 'region': 'Asia', 'category': 'Exotic'},
            'SYP': {'name': 'Syrian Pound', 'country': 'Syria', 'region': 'Asia', 'category': 'Exotic'},
            'LBP': {'name': 'Lebanese Pound', 'country': 'Lebanon', 'region': 'Asia', 'category': 'Exotic'},
            'ILS': {'name': 'Israeli Shekel', 'country': 'Israel', 'region': 'Asia', 'category': 'Exotic'},
            'TRY': {'name': 'Turkish Lira', 'country': 'Turkey', 'region': 'Asia/Europe', 'category': 'Exotic'},
            'GEL': {'name': 'Georgian Lari', 'country': 'Georgia', 'region': 'Asia', 'category': 'Exotic'},
            'AMD': {'name': 'Armenian Dram', 'country': 'Armenia', 'region': 'Asia', 'category': 'Exotic'},
            'AZN': {'name': 'Azerbaijani Manat', 'country': 'Azerbaijan', 'region': 'Asia', 'category': 'Exotic'},
            
            # African Currencies
            'ZAR': {'name': 'South African Rand', 'country': 'South Africa', 'region': 'Africa', 'category': 'Minor'},
            'EGP': {'name': 'Egyptian Pound', 'country': 'Egypt', 'region': 'Africa', 'category': 'Exotic'},
            'NGN': {'name': 'Nigerian Naira', 'country': 'Nigeria', 'region': 'Africa', 'category': 'Exotic'},
            'KES': {'name': 'Kenyan Shilling', 'country': 'Kenya', 'region': 'Africa', 'category': 'Exotic'},
            'GHS': {'name': 'Ghanaian Cedi', 'country': 'Ghana', 'region': 'Africa', 'category': 'Exotic'},
            'TND': {'name': 'Tunisian Dinar', 'country': 'Tunisia', 'region': 'Africa', 'category': 'Exotic'},
            'MAD': {'name': 'Moroccan Dirham', 'country': 'Morocco', 'region': 'Africa', 'category': 'Exotic'},
            'DZD': {'name': 'Algerian Dinar', 'country': 'Algeria', 'region': 'Africa', 'category': 'Exotic'},
            'LYD': {'name': 'Libyan Dinar', 'country': 'Libya', 'region': 'Africa', 'category': 'Exotic'},
            'ETB': {'name': 'Ethiopian Birr', 'country': 'Ethiopia', 'region': 'Africa', 'category': 'Exotic'},
            'UGX': {'name': 'Ugandan Shilling', 'country': 'Uganda', 'region': 'Africa', 'category': 'Exotic'},
            'TZS': {'name': 'Tanzanian Shilling', 'country': 'Tanzania', 'region': 'Africa', 'category': 'Exotic'},
            'RWF': {'name': 'Rwandan Franc', 'country': 'Rwanda', 'region': 'Africa', 'category': 'Exotic'},
            'BIF': {'name': 'Burundian Franc', 'country': 'Burundi', 'region': 'Africa', 'category': 'Exotic'},
            'DJF': {'name': 'Djiboutian Franc', 'country': 'Djibouti', 'region': 'Africa', 'category': 'Exotic'},
            'SOS': {'name': 'Somali Shilling', 'country': 'Somalia', 'region': 'Africa', 'category': 'Exotic'},
            'ERN': {'name': 'Eritrean Nakfa', 'country': 'Eritrea', 'region': 'Africa', 'category': 'Exotic'},
            'SDG': {'name': 'Sudanese Pound', 'country': 'Sudan', 'region': 'Africa', 'category': 'Exotic'},
            'SSP': {'name': 'South Sudanese Pound', 'country': 'South Sudan', 'region': 'Africa', 'category': 'Exotic'},
            'CDF': {'name': 'Congolese Franc', 'country': 'Democratic Republic of Congo', 'region': 'Africa', 'category': 'Exotic'},
            'XAF': {'name': 'Central African CFA Franc', 'country': 'Central African Economic Union', 'region': 'Africa', 'category': 'Exotic'},
            'XOF': {'name': 'West African CFA Franc', 'country': 'West African Economic Union', 'region': 'Africa', 'category': 'Exotic'},
            'AOA': {'name': 'Angolan Kwanza', 'country': 'Angola', 'region': 'Africa', 'category': 'Exotic'},
            'ZMW': {'name': 'Zambian Kwacha', 'country': 'Zambia', 'region': 'Africa', 'category': 'Exotic'},
            'ZWL': {'name': 'Zimbabwean Dollar', 'country': 'Zimbabwe', 'region': 'Africa', 'category': 'Exotic'},
            'BWP': {'name': 'Botswanan Pula', 'country': 'Botswana', 'region': 'Africa', 'category': 'Exotic'},
            'NAD': {'name': 'Namibian Dollar', 'country': 'Namibia', 'region': 'Africa', 'category': 'Exotic'},
            'SZL': {'name': 'Swazi Lilangeni', 'country': 'Eswatini', 'region': 'Africa', 'category': 'Exotic'},
            'LSL': {'name': 'Lesotho Loti', 'country': 'Lesotho', 'region': 'Africa', 'category': 'Exotic'},
            'MWK': {'name': 'Malawian Kwacha', 'country': 'Malawi', 'region': 'Africa', 'category': 'Exotic'},
            'MZN': {'name': 'Mozambican Metical', 'country': 'Mozambique', 'region': 'Africa', 'category': 'Exotic'},
            'MGA': {'name': 'Malagasy Ariary', 'country': 'Madagascar', 'region': 'Africa', 'category': 'Exotic'},
            'MUR': {'name': 'Mauritian Rupee', 'country': 'Mauritius', 'region': 'Africa', 'category': 'Exotic'},
            'SCR': {'name': 'Seychellois Rupee', 'country': 'Seychelles', 'region': 'Africa', 'category': 'Exotic'},
            'KMF': {'name': 'Comorian Franc', 'country': 'Comoros', 'region': 'Africa', 'category': 'Exotic'},
            'GMD': {'name': 'Gambian Dalasi', 'country': 'Gambia', 'region': 'Africa', 'category': 'Exotic'},
            'GNF': {'name': 'Guinean Franc', 'country': 'Guinea', 'region': 'Africa', 'category': 'Exotic'},
            'GWP': {'name': 'Guinea-Bissau Peso', 'country': 'Guinea-Bissau', 'region': 'Africa', 'category': 'Exotic'},
            'SLL': {'name': 'Sierra Leonean Leone', 'country': 'Sierra Leone', 'region': 'Africa', 'category': 'Exotic'},
            'LRD': {'name': 'Liberian Dollar', 'country': 'Liberia', 'region': 'Africa', 'category': 'Exotic'},
            'CVE': {'name': 'Cape Verdean Escudo', 'country': 'Cape Verde', 'region': 'Africa', 'category': 'Exotic'},
            'STN': {'name': 'São Tomé and Príncipe Dobra', 'country': 'São Tomé and Príncipe', 'region': 'Africa', 'category': 'Exotic'},
            
            # Americas Currencies
            'MXN': {'name': 'Mexican Peso', 'country': 'Mexico', 'region': 'North America', 'category': 'Minor'},
            'BRL': {'name': 'Brazilian Real', 'country': 'Brazil', 'region': 'South America', 'category': 'Minor'},
            'ARS': {'name': 'Argentine Peso', 'country': 'Argentina', 'region': 'South America', 'category': 'Exotic'},
            'CLP': {'name': 'Chilean Peso', 'country': 'Chile', 'region': 'South America', 'category': 'Exotic'},
            'COP': {'name': 'Colombian Peso', 'country': 'Colombia', 'region': 'South America', 'category': 'Exotic'},
            'PEN': {'name': 'Peruvian Sol', 'country': 'Peru', 'region': 'South America', 'category': 'Exotic'},
            'UYU': {'name': 'Uruguayan Peso', 'country': 'Uruguay', 'region': 'South America', 'category': 'Exotic'},
            'PYG': {'name': 'Paraguayan Guarani', 'country': 'Paraguay', 'region': 'South America', 'category': 'Exotic'},
            'BOB': {'name': 'Bolivian Boliviano', 'country': 'Bolivia', 'region': 'South America', 'category': 'Exotic'},
            'VES': {'name': 'Venezuelan Bolívar', 'country': 'Venezuela', 'region': 'South America', 'category': 'Exotic'},
            'GYD': {'name': 'Guyanese Dollar', 'country': 'Guyana', 'region': 'South America', 'category': 'Exotic'},
            'SRD': {'name': 'Surinamese Dollar', 'country': 'Suriname', 'region': 'South America', 'category': 'Exotic'},
            'GTQ': {'name': 'Guatemalan Quetzal', 'country': 'Guatemala', 'region': 'Central America', 'category': 'Exotic'},
            'CRC': {'name': 'Costa Rican Colón', 'country': 'Costa Rica', 'region': 'Central America', 'category': 'Exotic'},
            'NIO': {'name': 'Nicaraguan Córdoba', 'country': 'Nicaragua', 'region': 'Central America', 'category': 'Exotic'},
            'HNL': {'name': 'Honduran Lempira', 'country': 'Honduras', 'region': 'Central America', 'category': 'Exotic'},
            'BZD': {'name': 'Belize Dollar', 'country': 'Belize', 'region': 'Central America', 'category': 'Exotic'},
            'SVC': {'name': 'Salvadoran Colón', 'country': 'El Salvador', 'region': 'Central America', 'category': 'Exotic'},
            'PAB': {'name': 'Panamanian Balboa', 'country': 'Panama', 'region': 'Central America', 'category': 'Exotic'},
            'DOP': {'name': 'Dominican Peso', 'country': 'Dominican Republic', 'region': 'Caribbean', 'category': 'Exotic'},
            'HTG': {'name': 'Haitian Gourde', 'country': 'Haiti', 'region': 'Caribbean', 'category': 'Exotic'},
            'JMD': {'name': 'Jamaican Dollar', 'country': 'Jamaica', 'region': 'Caribbean', 'category': 'Exotic'},
            'CUP': {'name': 'Cuban Peso', 'country': 'Cuba', 'region': 'Caribbean', 'category': 'Exotic'},
            'BSD': {'name': 'Bahamian Dollar', 'country': 'Bahamas', 'region': 'Caribbean', 'category': 'Exotic'},
            'BBD': {'name': 'Barbadian Dollar', 'country': 'Barbados', 'region': 'Caribbean', 'category': 'Exotic'},
            'TTD': {'name': 'Trinidad and Tobago Dollar', 'country': 'Trinidad and Tobago', 'region': 'Caribbean', 'category': 'Exotic'},
            'XCD': {'name': 'East Caribbean Dollar', 'country': 'Eastern Caribbean', 'region': 'Caribbean', 'category': 'Exotic'},
            'AWG': {'name': 'Aruban Florin', 'country': 'Aruba', 'region': 'Caribbean', 'category': 'Exotic'},
            'ANG': {'name': 'Netherlands Antillean Guilder', 'country': 'Curaçao and Sint Maarten', 'region': 'Caribbean', 'category': 'Exotic'},
            
            # Oceania Currencies
            'FJD': {'name': 'Fijian Dollar', 'country': 'Fiji', 'region': 'Oceania', 'category': 'Exotic'},
            'PGK': {'name': 'Papua New Guinean Kina', 'country': 'Papua New Guinea', 'region': 'Oceania', 'category': 'Exotic'},
            'SBD': {'name': 'Solomon Islands Dollar', 'country': 'Solomon Islands', 'region': 'Oceania', 'category': 'Exotic'},
            'VUV': {'name': 'Vanuatu Vatu', 'country': 'Vanuatu', 'region': 'Oceania', 'category': 'Exotic'},
            'NCR': {'name': 'New Caledonian Franc', 'country': 'New Caledonia', 'region': 'Oceania', 'category': 'Exotic'},
            'XPF': {'name': 'CFP Franc', 'country': 'French Polynesia', 'region': 'Oceania', 'category': 'Exotic'},
            'WST': {'name': 'Samoan Tala', 'country': 'Samoa', 'region': 'Oceania', 'category': 'Exotic'},
            'TOP': {'name': 'Tongan Paʻanga', 'country': 'Tonga', 'region': 'Oceania', 'category': 'Exotic'},
            'KID': {'name': 'Kiribati Dollar', 'country': 'Kiribati', 'region': 'Oceania', 'category': 'Exotic'},
            'TVD': {'name': 'Tuvaluan Dollar', 'country': 'Tuvalu', 'region': 'Oceania', 'category': 'Exotic'},
            'NRU': {'name': 'Nauruan Dollar', 'country': 'Nauru', 'region': 'Oceania', 'category': 'Exotic'},
            'MHL': {'name': 'Marshallese Dollar', 'country': 'Marshall Islands', 'region': 'Oceania', 'category': 'Exotic'},
            'FSM': {'name': 'Micronesian Dollar', 'country': 'Federated States of Micronesia', 'region': 'Oceania', 'category': 'Exotic'},
            'PLW': {'name': 'Palauan Dollar', 'country': 'Palau', 'region': 'Oceania', 'category': 'Exotic'},
        }
        
        self.logger.info(f"Loaded {len(self.world_currencies)} world currencies")
    
    def get_all_currencies(self) -> List[str]:
        """Get list of all currency codes"""
        return list(self.world_currencies.keys())
    
    def get_currencies_by_category(self, category: str) -> List[str]:
        """Get currencies by category (Major, Minor, Exotic)"""
        return [code for code, info in self.world_currencies.items() 
                if info['category'] == category]
    
    def get_currencies_by_region(self, region: str) -> List[str]:
        """Get currencies by geographical region"""
        return [code for code, info in self.world_currencies.items() 
                if info['region'] == region]
    
    def get_currency_info(self, currency_code: str) -> Dict:
        """Get detailed information about a currency"""
        return self.world_currencies.get(currency_code, {})
    
    def get_trading_base_currencies(self) -> List[str]:
        """Get commonly used base currencies for forex trading"""
        return ['USD', 'EUR', 'GBP', 'JPY', 'CHF', 'CAD', 'AUD', 'NZD']
    
    def generate_currency_pairs(self, base_currencies: List[str] = None) -> List[tuple]:
        """Generate all possible currency pairs"""
        if base_currencies is None:
            base_currencies = self.get_trading_base_currencies()
        
        all_currencies = self.get_all_currencies()
        pairs = []
        
        for base in base_currencies:
            for quote in all_currencies:
                if base != quote:
                    pairs.append((base, quote))
        
        return pairs
    
    def print_currency_summary(self):
        """Print summary of all currencies by region and category"""
        print(f"\n🌍 WORLD CURRENCIES DATABASE")
        print("=" * 60)
        print(f"Total currencies: {len(self.world_currencies)}")
        
        # By category
        categories = {}
        for info in self.world_currencies.values():
            cat = info['category']
            categories[cat] = categories.get(cat, 0) + 1
        
        print(f"\nBy Category:")
        for category, count in categories.items():
            print(f"  {category:<10}: {count:>3} currencies")
        
        # By region
        regions = {}
        for info in self.world_currencies.values():
            region = info['region']
            regions[region] = regions.get(region, 0) + 1
        
        print(f"\nBy Region:")
        for region, count in sorted(regions.items()):
            print(f"  {region:<20}: {count:>3} currencies")

# Create world currencies database
world_currencies = WorldCurrenciesDatabase()
world_currencies.print_currency_summary()
print("✅ World currencies database created")

### Connect to Live Environment

In [ ]:
# Create and connect to live trading environment
live_connection = LiveForexConnection()

print("🔌 Connecting to Interactive Brokers live environment...")
print("⚠️  WARNING: This will connect to LIVE trading environment!")
print("   Make sure TWS/Gateway is running on port 7496")

connection_success = live_connection.connect()

if connection_success:
    # Get account information
    account_info = live_connection.get_account_info()
    print(f"\n📊 Account Information:")
    if account_info.get('accounts'):
        print(f"   Connected accounts: {', '.join(account_info['accounts'])}")
    print(f"   Environment: {account_info.get('environment', 'Unknown')}")
    print(f"   Status: Ready for live forex trading")
else:
    print("\n❌ Failed to connect to live environment")
    print("   Please ensure:")
    print("   1. TWS or IB Gateway is running")
    print("   2. API connections are enabled")
    print("   3. Port 7496 is configured for live trading")
    print("   4. Client ID 1 is available")

### World Forex Pair Discovery Class

In [ ]:
class WorldForexDiscovery:
    """Discovers all available forex pairs from world currencies"""
    
    def __init__(self, connection: LiveForexConnection, currencies_db: WorldCurrenciesDatabase):
        """Initialise with live connection and currencies database"""
        self.connection = connection
        self.ib = connection.ib
        self.currencies_db = currencies_db
        self.logger = logging.getLogger('WorldForexDiscovery')
        
        # Track discovered pairs
        self.available_pairs = []
        self.unavailable_pairs = []
        
    def discover_all_pairs(self, base_currencies: List[str] = None) -> List[Dict]:
        """Discover all available forex pairs for given base currencies"""
        if not self.connection.is_connected():
            self.logger.error("Not connected to IB live environment")
            return []
        
        if base_currencies is None:
            base_currencies = self.currencies_db.get_trading_base_currencies()
        
        print(f"\n🔍 Discovering forex pairs for {len(base_currencies)} base currencies...")
        print(f"Base currencies: {', '.join(base_currencies)}")
        
        all_quote_currencies = self.currencies_db.get_all_currencies()
        total_combinations = len(base_currencies) * (len(all_quote_currencies) - 1)
        
        print(f"Testing {total_combinations} potential currency pairs...")
        print("This may take several minutes...\n")
        
        discovered_pairs = []
        processed = 0
        
        for base in base_currencies:
            print(f"Testing {base} pairs...")
            
            for quote in all_quote_currencies:
                if base == quote:
                    continue
                
                processed += 1
                
                try:
                    # Create forex contract
                    pair_symbol = f"{base}{quote}"
                    contract = Forex(pair_symbol)
                    
                    # Request contract details to validate
                    details = self.ib.reqContractDetails(contract)
                    
                    if details:
                        base_info = self.currencies_db.get_currency_info(base)
                        quote_info = self.currencies_db.get_currency_info(quote)
                        
                        pair_info = {
                            'symbol': pair_symbol,
                            'base_currency': base,
                            'quote_currency': quote,
                            'base_name': base_info.get('name', base),
                            'quote_name': quote_info.get('name', quote),
                            'base_country': base_info.get('country', 'Unknown'),
                            'quote_country': quote_info.get('country', 'Unknown'),
                            'base_region': base_info.get('region', 'Unknown'),
                            'quote_region': quote_info.get('region', 'Unknown'),
                            'category': self._determine_pair_category(base_info, quote_info),
                            'contract': contract,
                            'tradeable': True,
                            'pip_size': self._get_pip_size(quote),
                            'contract_details': details[0] if details else None
                        }
                        
                        discovered_pairs.append(pair_info)
                        self.available_pairs.append(pair_symbol)
                        
                        print(f"✅ {pair_symbol:<8} - {base_info.get('name', base)} / {quote_info.get('name', quote)}")
                    else:
                        self.unavailable_pairs.append(f"{base}{quote}")
                
                except Exception as e:
                    self.unavailable_pairs.append(f"{base}{quote}")
                    if "No security definition" not in str(e):
                        self.logger.warning(f"Error checking {base}/{quote}: {e}")
                
                # Progress indicator
                if processed % 50 == 0:
                    print(f"   Progress: {processed}/{total_combinations} ({processed/total_combinations*100:.1f}%)")
                
                # Small delay to avoid rate limiting
                time.sleep(0.05)
        
        print(f"\n📊 Discovery Complete!")
        print(f"   Available pairs: {len(discovered_pairs)}")
        print(f"   Unavailable pairs: {len(self.unavailable_pairs)}")
        print(f"   Success rate: {len(discovered_pairs)/(len(discovered_pairs)+len(self.unavailable_pairs))*100:.1f}%")
        
        return discovered_pairs
    
    def _determine_pair_category(self, base_info: Dict, quote_info: Dict) -> str:
        """Determine pair category based on currency information"""
        base_cat = base_info.get('category', 'Exotic')
        quote_cat = quote_info.get('category', 'Exotic')
        
        # Both major currencies = Major pair
        if base_cat == 'Major' and quote_cat == 'Major':
            return 'Major'
        # One major, one minor = Minor pair
        elif (base_cat in ['Major', 'Minor'] and quote_cat in ['Major', 'Minor']):
            return 'Minor'
        # Everything else = Exotic
        else:
            return 'Exotic'
    
    def _get_pip_size(self, quote_currency: str) -> float:
        """Get pip size based on quote currency"""
        # JPY pairs typically have 2 decimal places
        if quote_currency == 'JPY':
            return 0.01
        # HUF, KRW, etc. have different pip sizes
        elif quote_currency in ['HUF', 'KRW', 'JPY', 'CLP', 'IDR', 'KRW']:
            return 0.01
        # Most pairs have 4 decimal places
        else:
            return 0.0001
    
    def get_market_data_sample(self, pairs: List[Dict], max_pairs: int = 20) -> pd.DataFrame:
        """Get market data for a sample of pairs"""
        if not pairs:
            self.logger.error("No forex pairs provided")
            return pd.DataFrame()
        
        # Sample pairs for market data (to avoid overwhelming the system)
        sample_pairs = pairs[:max_pairs]
        print(f"\n📈 Fetching market data for {len(sample_pairs)} sample pairs...")
        
        market_data = []
        
        for i, pair in enumerate(sample_pairs):
            try:
                contract = pair['contract']
                
                # Request market data
                ticker = self.ib.reqMktData(contract, '', False, False)
                self.ib.sleep(2)  # Wait for data
                
                if ticker and (ticker.bid or ticker.ask or ticker.last):
                    # Calculate spread in pips
                    spread_pips = None
                    if ticker.bid and ticker.ask and ticker.bid > 0 and ticker.ask > 0:
                        spread_pips = (ticker.ask - ticker.bid) / pair['pip_size']
                    
                    data = {
                        'symbol': pair['symbol'],
                        'base': pair['base_currency'],
                        'quote': pair['quote_currency'],
                        'base_name': pair['base_name'],
                        'quote_name': pair['quote_name'],
                        'category': pair['category'],
                        'bid': ticker.bid if ticker.bid and ticker.bid > 0 else None,
                        'ask': ticker.ask if ticker.ask and ticker.ask > 0 else None,
                        'last': ticker.last if ticker.last and ticker.last > 0 else None,
                        'spread_pips': spread_pips,
                        'pip_size': pair['pip_size'],
                        'timestamp': datetime.now()
                    }
                    market_data.append(data)
                    print(f"✅ {pair['symbol']:<8} - Bid: {ticker.bid:.5f}, Ask: {ticker.ask:.5f}")
                
                # Cancel market data subscription
                self.ib.cancelMktData(contract)
                
            except Exception as e:
                self.logger.error(f"Error getting market data for {pair['symbol']}: {e}")
            
            # Progress indicator
            if (i + 1) % 5 == 0:
                print(f"   Processed {i + 1}/{len(sample_pairs)} pairs")
        
        return pd.DataFrame(market_data) if market_data else pd.DataFrame()
    
    def display_discovery_summary(self, pairs: List[Dict]):
        """Display comprehensive summary of discovered pairs"""
        if not pairs:
            print("No forex pairs discovered")
            return
        
        print(f"\n🌍 WORLD FOREX PAIRS DISCOVERY SUMMARY")
        print("=" * 80)
        print(f"Total discovered pairs: {len(pairs)}")
        
        # Category breakdown
        categories = {}
        for pair in pairs:
            cat = pair['category']
            categories[cat] = categories.get(cat, 0) + 1
        
        print(f"\nBy Category:")
        for category, count in categories.items():
            print(f"  {category:<10}: {count:>4} pairs")
        
        # Regional breakdown
        regions = {}
        for pair in pairs:
            base_region = pair['base_region']
            quote_region = pair['quote_region']
            
            # Count base currency regions
            regions[f"{base_region} (base)"] = regions.get(f"{base_region} (base)", 0) + 1
            
            # Count quote currency regions
            if base_region != quote_region:
                regions[f"{quote_region} (quote)"] = regions.get(f"{quote_region} (quote)", 0) + 1
        
        print(f"\nBy Region:")
        for region, count in sorted(regions.items()):
            print(f"  {region:<25}: {count:>4} pairs")
        
        # Base currency breakdown
        base_currencies = {}
        for pair in pairs:
            base = pair['base_currency']
            base_currencies[base] = base_currencies.get(base, 0) + 1
        
        print(f"\nTop Base Currencies:")
        sorted_bases = sorted(base_currencies.items(), key=lambda x: x[1], reverse=True)
        for base, count in sorted_bases[:15]:
            base_name = self.currencies_db.get_currency_info(base).get('name', base)
            print(f"  {base} ({base_name:<20}): {count:>3} pairs")

print("✅ World forex discovery class created")

### Discover All Available World Forex Pairs

In [ ]:
# Start comprehensive forex pair discovery
if live_connection.is_connected():
    print("🚀 Starting comprehensive world forex discovery...")
    print("⏱️  This process may take 10-15 minutes to complete")
    
    # Create discovery instance
    discovery = WorldForexDiscovery(live_connection, world_currencies)
    
    # Get base currencies for discovery (you can customize this list)
    base_currencies = world_currencies.get_trading_base_currencies()
    
    # Optional: Add more base currencies for broader discovery
    additional_bases = ['CNY', 'HKD', 'SGD', 'KRW', 'INR', 'ZAR', 'MXN', 'BRL', 'TRY']
    base_currencies.extend(additional_bases)
    base_currencies = list(set(base_currencies))  # Remove duplicates
    
    print(f"Using {len(base_currencies)} base currencies: {', '.join(base_currencies)}")
    
    # Discover all available pairs
    start_time = datetime.now()
    world_forex_pairs = discovery.discover_all_pairs(base_currencies)
    end_time = datetime.now()
    
    duration = end_time - start_time
    print(f"\n⏱️  Discovery completed in {duration.total_seconds():.1f} seconds")
    
    # Display comprehensive summary
    discovery.display_discovery_summary(world_forex_pairs)
    
    # Store for later use
    forex_pairs_data = world_forex_pairs
    
else:
    print("❌ Cannot start discovery - not connected to live environment")
    forex_pairs_data = []

### Get Market Data Sample

In [ ]:
# Get real-time market data for a sample of discovered pairs
if forex_pairs_data and live_connection.is_connected():
    print("\n📊 Fetching real-time market data sample...")
    
    # Get market data for top pairs from each category
    major_pairs = [p for p in forex_pairs_data if p['category'] == 'Major'][:8]
    minor_pairs = [p for p in forex_pairs_data if p['category'] == 'Minor'][:8]
    exotic_pairs = [p for p in forex_pairs_data if p['category'] == 'Exotic'][:10]
    
    sample_pairs = major_pairs + minor_pairs + exotic_pairs
    
    market_df = discovery.get_market_data_sample(sample_pairs, max_pairs=25)
    
    if not market_df.empty:
        # Display formatted results
        print(f"\n💰 Live Market Data Sample ({len(market_df)} pairs):")
        print("=" * 120)
        print(f"{'Symbol':<10} {'Category':<8} {'Base Currency':<20} {'Quote Currency':<20} {'Bid':<12} {'Ask':<12} {'Spread':<10}")
        print("-" * 120)
        
        for idx, row in market_df.iterrows():
            bid_str = f"{row['bid']:.5f}" if pd.notna(row['bid']) else "N/A"
            ask_str = f"{row['ask']:.5f}" if pd.notna(row['ask']) else "N/A"
            spread_str = f"{row['spread_pips']:.1f} pips" if pd.notna(row['spread_pips']) else "N/A"
            
            print(f"{row['symbol']:<10} {row['category']:<8} {row['base_name']:<20} {row['quote_name']:<20} {bid_str:<12} {ask_str:<12} {spread_str:<10}")
        
        # Save market data
        market_df.to_csv('world_forex_market_data.csv', index=False)
        print(f"\n✅ Market data saved to world_forex_market_data.csv")
        
    else:
        print("❌ No market data available")
else:
    print("Cannot fetch market data - no pairs discovered or connection unavailable")

### Get Market Data Large Sample

In [ ]:
# Get real-time market data for 200 pairs sample
if forex_pairs_data and live_connection.is_connected():
    print("\n📊 Fetching real-time market data for 200 pairs...")
    
    # Get more pairs from each category to reach 200 total
    major_pairs = [p for p in forex_pairs_data if p['category'] == 'Major'][:50]  # Up to 50 major pairs
    minor_pairs = [p for p in forex_pairs_data if p['category'] == 'Minor'][:75]  # Up to 75 minor pairs
    exotic_pairs = [p for p in forex_pairs_data if p['category'] == 'Exotic'][:75]  # Up to 75 exotic pairs
    
    # Combine all pairs
    sample_pairs = major_pairs + minor_pairs + exotic_pairs
    
    # Ensure we don't exceed available pairs and limit to 200
    sample_pairs = sample_pairs[:200]
    
    print(f"Selected {len(sample_pairs)} pairs for market data:")
    print(f"  Major pairs: {len(major_pairs)}")
    print(f"  Minor pairs: {len(minor_pairs)}")
    print(f"  Exotic pairs: {len(exotic_pairs)}")
    
    # Process in batches to avoid overwhelming the API
    batch_size = 25  # Keep individual batch size manageable
    all_market_data = []
    total_batches = (len(sample_pairs) + batch_size - 1) // batch_size
    
    print(f"Processing {len(sample_pairs)} pairs in {total_batches} batches of {batch_size}...")
    
    for batch_num in range(total_batches):
        batch_start = batch_num * batch_size
        batch_end = min(batch_start + batch_size, len(sample_pairs))
        batch_pairs = sample_pairs[batch_start:batch_end]
        
        print(f"Processing batch {batch_num + 1}/{total_batches}: pairs {batch_start + 1}-{batch_end}")
        
        try:
            # Get market data for this batch
            batch_market_df = discovery.get_market_data_sample(batch_pairs, max_pairs=batch_size)
            
            if not batch_market_df.empty:
                all_market_data.append(batch_market_df)
                print(f"✅ Successfully fetched data for {len(batch_market_df)} pairs in batch {batch_num + 1}")
            else:
                print(f"⚠️ No data returned for batch {batch_num + 1}")
        
        except Exception as e:
            print(f"❌ Error processing batch {batch_num + 1}: {e}")
            continue
        
        # Small delay between batches to be respectful to the API
        time.sleep(1)
    
    # Combine all market data from batches
    if all_market_data:
        market_df = pd.concat(all_market_data, ignore_index=True)
        
        # Sort by category for better display
        category_order = {'Major': 1, 'Minor': 2, 'Exotic': 3}
        market_df['category_sort'] = market_df['category'].map(category_order)
        market_df = market_df.sort_values(['category_sort', 'symbol']).drop('category_sort', axis=1)
        
        if not market_df.empty:
            # Display formatted results (show first 50 for readability)
            display_count = min(50, len(market_df))
            print(f"\n💰 Live Market Data Sample (showing first {display_count} of {len(market_df)} pairs):")
            print("=" * 120)
            print(f"{'Symbol':<10} {'Category':<8} {'Base Currency':<20} {'Quote Currency':<20} {'Bid':<12} {'Ask':<12} {'Spread':<10}")
            print("-" * 120)
            
            for idx, row in market_df.head(display_count).iterrows():
                bid_str = f"{row['bid']:.5f}" if pd.notna(row['bid']) else "N/A"
                ask_str = f"{row['ask']:.5f}" if pd.notna(row['ask']) else "N/A"
                spread_str = f"{row['spread_pips']:.1f} pips" if pd.notna(row['spread_pips']) else "N/A"
                
                print(f"{row['symbol']:<10} {row['category']:<8} {row['base_name']:<20} {row['quote_name']:<20} {bid_str:<12} {ask_str:<12} {spread_str:<10}")
            
            if len(market_df) > display_count:
                print(f"... and {len(market_df) - display_count} more pairs")
            
            # Save complete market data
            market_df.to_csv('world_forex_market_data_200.csv', index=False)
            print(f"\n✅ Complete market data for {len(market_df)} pairs saved to world_forex_market_data_200.csv")
            
            # Create summary statistics
            print(f"\n📊 Market Data Summary:")
            print(f"  Total pairs with data: {len(market_df)}")
            print(f"  Pairs with valid bid/ask: {market_df[['bid', 'ask']].notna().all(axis=1).sum()}")
            
            # Category breakdown
            category_breakdown = market_df['category'].value_counts()
            for category, count in category_breakdown.items():
                percentage = (count / len(market_df)) * 100
                print(f"  {category} pairs: {count} ({percentage:.1f}%)")
            
            # Spread analysis (for pairs with valid spreads)
            valid_spreads = market_df.dropna(subset=['spread_pips'])
            if not valid_spreads.empty:
                print(f"\n📈 Spread Analysis ({len(valid_spreads)} pairs with valid spreads):")
                print(f"  Average spread: {valid_spreads['spread_pips'].mean():.2f} pips")
                print(f"  Median spread: {valid_spreads['spread_pips'].median():.2f} pips")
                print(f"  Min spread: {valid_spreads['spread_pips'].min():.2f} pips")
                print(f"  Max spread: {valid_spreads['spread_pips'].max():.2f} pips")
                
                # Best and worst spreads
                best_spreads = valid_spreads.nsmallest(5, 'spread_pips')
                worst_spreads = valid_spreads.nlargest(5, 'spread_pips')
                
                print(f"\n🎯 Best Spreads (Top 5):")
                for _, pair in best_spreads.iterrows():
                    print(f"  {pair['symbol']:<8} ({pair['category']:<6}): {pair['spread_pips']:.1f} pips")
                
                print(f"\n⚠️ Highest Spreads (Top 5):")
                for _, pair in worst_spreads.iterrows():
                    print(f"  {pair['symbol']:<8} ({pair['category']:<6}): {pair['spread_pips']:.1f} pips")
            
            # Export by category with market data
            for category in ['Major', 'Minor', 'Exotic']:
                category_df = market_df[market_df['category'] == category]
                if not category_df.empty:
                    filename = f'market_data_{category.lower()}_pairs.csv'
                    category_df.to_csv(filename, index=False)
                    print(f"✅ {category} pairs market data ({len(category_df)}) saved to {filename}")
            
        else:
            print("❌ No market data available after processing all batches")
    else:
        print("❌ No market data collected from any batch")
        
else:
    print("Cannot fetch market data - no pairs discovered or connection unavailable")

# Additional analysis if we successfully got 200 pairs of market data
if 'market_df' in locals() and not market_df.empty and len(market_df) >= 100:
    print(f"\n🔍 EXTENDED ANALYSIS FOR {len(market_df)} PAIRS:")
    print("=" * 60)
    
    # Regional analysis
    if 'base_region' in market_df.columns:
        region_analysis = market_df['base_region'].value_counts().head(10)
        print(f"\nTop Regions by Pair Count:")
        for region, count in region_analysis.items():
            print(f"  {region:<20}: {count:>3} pairs")
    
    # Base currency analysis
    if 'base' in market_df.columns:
        base_analysis = market_df['base'].value_counts().head(10)
        print(f"\nTop Base Currencies:")
        for base, count in base_analysis.items():
            print(f"  {base:<5}: {count:>3} pairs")
    
    # Liquidity indicators (based on spread)
    if 'spread_pips' in market_df.columns:
        valid_spreads = market_df.dropna(subset=['spread_pips'])
        if len(valid_spreads) >= 20:
            # Define liquidity tiers based on spreads
            high_liquidity = valid_spreads[valid_spreads['spread_pips'] <= 3.0]
            medium_liquidity = valid_spreads[(valid_spreads['spread_pips'] > 3.0) & (valid_spreads['spread_pips'] <= 10.0)]
            low_liquidity = valid_spreads[valid_spreads['spread_pips'] > 10.0]
            
            print(f"\nLiquidity Tiers:")
            print(f"  High Liquidity (≤3 pips): {len(high_liquidity)} pairs")
            print(f"  Medium Liquidity (3-10 pips): {len(medium_liquidity)} pairs")  
            print(f"  Low Liquidity (>10 pips): {len(low_liquidity)} pairs")
            
            # Show examples from each tier
            if not high_liquidity.empty:
                print(f"\n  High Liquidity Examples:")
                for _, pair in high_liquidity.head(5).iterrows():
                    print(f"    {pair['symbol']:<8}: {pair['spread_pips']:.1f} pips")
            
            if not medium_liquidity.empty:
                print(f"\n  Medium Liquidity Examples:")
                for _, pair in medium_liquidity.head(5).iterrows():
                    print(f"    {pair['symbol']:<8}: {pair['spread_pips']:.1f} pips")
            
            if not low_liquidity.empty:
                print(f"\n  Low Liquidity Examples:")
                for _, pair in low_liquidity.head(5).iterrows():
                    print(f"    {pair['symbol']:<8}: {pair['spread_pips']:.1f} pips")
    
    # Trading session analysis for the 200 pairs
    if 'base' in market_df.columns and 'quote' in market_df.columns:
        print(f"\nOptimal Trading Sessions for 200-Pair Sample:")
        
        # Asian session currencies
        asian_currencies = ['JPY', 'AUD', 'NZD', 'CNY', 'HKD', 'SGD', 'KRW', 'TWD', 'INR', 'IDR', 'MYR', 'THB', 'PHP']
        asian_pairs = market_df[
            (market_df['base'].isin(asian_currencies)) | 
            (market_df['quote'].isin(asian_currencies))
        ]
        
        # European session currencies  
        european_currencies = ['EUR', 'GBP', 'CHF', 'SEK', 'NOK', 'DKK', 'PLN', 'CZK', 'HUF', 'TRY']
        european_pairs = market_df[
            (market_df['base'].isin(european_currencies)) | 
            (market_df['quote'].isin(european_currencies))
        ]
        
        # American session currencies
        american_currencies = ['USD', 'CAD', 'MXN', 'BRL', 'ARS', 'CLP', 'COP', 'PEN']
        american_pairs = market_df[
            (market_df['base'].isin(american_currencies)) | 
            (market_df['quote'].isin(american_currencies))
        ]
        
        print(f"  Asian Session (21:00-06:00 GMT): {len(asian_pairs)} pairs")
        print(f"  European Session (07:00-16:00 GMT): {len(european_pairs)} pairs")
        print(f"  American Session (13:00-22:00 GMT): {len(american_pairs)} pairs")
    
    # Risk distribution analysis
    print(f"\nRisk Profile Distribution:")
    if 'category' in market_df.columns:
        risk_profiles = {
            'Conservative (Major pairs)': len(market_df[market_df['category'] == 'Major']),
            'Moderate (Minor pairs)': len(market_df[market_df['category'] == 'Minor']),
            'Aggressive (Exotic pairs)': len(market_df[market_df['category'] == 'Exotic'])
        }
        
        for profile, count in risk_profiles.items():
            percentage = (count / len(market_df)) * 100
            print(f"  {profile}: {count} pairs ({percentage:.1f}%)")
    
    print(f"\nRecommended Portfolio Allocation for 200-Pair Dataset:")
    print(f"  • Conservative traders: Focus on Major pairs (50-70% allocation)")
    print(f"  • Moderate traders: Mix Major + Minor pairs (30% Major, 40% Minor, 30% Exotic)")
    print(f"  • Aggressive traders: Include all categories with proper risk management")
    print(f"  • Never exceed 2% risk per trade regardless of pair type")
    print(f"  • Consider correlation - avoid trading too many similar pairs simultaneously")

### USD Base Pairs

In [ ]:
# Get real-time market data for USD base currency pairs to all world currencies
if forex_pairs_data and live_connection.is_connected():
    print("\n💱 Fetching USD-based pairs to all world currencies...")
    
    # Filter for USD base currency pairs only
    usd_pairs = [p for p in forex_pairs_data if p['base_currency'] == 'USD']
    
    print(f"Found {len(usd_pairs)} USD base currency pairs")
    
    # Group USD pairs by category for better organisation
    usd_major_pairs = [p for p in usd_pairs if p['category'] == 'Major']
    usd_minor_pairs = [p for p in usd_pairs if p['category'] == 'Minor']
    usd_exotic_pairs = [p for p in usd_pairs if p['category'] == 'Exotic']
    
    print(f"USD Major pairs: {len(usd_major_pairs)}")
    print(f"USD Minor pairs: {len(usd_minor_pairs)}")
    print(f"USD Exotic pairs: {len(usd_exotic_pairs)}")
    
    # Get market data for ALL USD pairs (not just sample)
    print(f"\n📈 Fetching real-time market data for all {len(usd_pairs)} USD pairs...")
    
    # Process in smaller batches to avoid overwhelming the system
    batch_size = 30
    all_usd_market_data = []
    
    for batch_start in range(0, len(usd_pairs), batch_size):
        batch_end = min(batch_start + batch_size, len(usd_pairs))
        batch_pairs = usd_pairs[batch_start:batch_end]
        
        print(f"Processing batch {batch_start//batch_size + 1}: pairs {batch_start + 1}-{batch_end}")
        
        batch_market_df = discovery.get_market_data_sample(batch_pairs, max_pairs=batch_size)
        
        if not batch_market_df.empty:
            all_usd_market_data.append(batch_market_df)
            print(f"✅ Successfully fetched data for {len(batch_market_df)} pairs in this batch")
        
        # Small delay between batches to be respectful to the API
        time.sleep(2)
    
    # Combine all market data
    if all_usd_market_data:
        usd_market_df = pd.concat(all_usd_market_data, ignore_index=True)
        
        # Sort by category and symbol for better presentation
        category_order = {'Major': 1, 'Minor': 2, 'Exotic': 3}
        usd_market_df['category_sort'] = usd_market_df['category'].map(category_order)
        usd_market_df = usd_market_df.sort_values(['category_sort', 'symbol']).drop('category_sort', axis=1)
        
        # Display formatted results
        print(f"\n💰 USD Live Market Data ({len(usd_market_df)} pairs):")
        print("=" * 140)
        print(f"{'Symbol':<12} {'Category':<8} {'Quote Currency':<25} {'Country':<25} {'Bid':<12} {'Ask':<12} {'Spread':<10}")
        print("-" * 140)
        
        for idx, row in usd_market_df.iterrows():
            bid_str = f"{row['bid']:.5f}" if pd.notna(row['bid']) else "N/A"
            ask_str = f"{row['ask']:.5f}" if pd.notna(row['ask']) else "N/A"
            spread_str = f"{row['spread_pips']:.1f} pips" if pd.notna(row['spread_pips']) else "N/A"
            
            # Get quote currency country info
            quote_currency = row['symbol'][3:]  # Extract quote currency from USD+XXX format
            currency_info = world_currencies.get_currency_info(quote_currency)
            country = currency_info.get('country', 'Unknown')[:24]  # Truncate long country names
            
            print(f"{row['symbol']:<12} {row['category']:<8} {row['quote_name']:<25} {country:<25} {bid_str:<12} {ask_str:<12} {spread_str:<10}")
        
        # Create enhanced USD pairs export with additional analysis
        usd_export_data = []
        for _, row in usd_market_df.iterrows():
            quote_currency = row['symbol'][3:]
            currency_info = world_currencies.get_currency_info(quote_currency)
            
            export_row = {
                'symbol': row['symbol'],
                'base_currency': 'USD',
                'quote_currency': quote_currency,
                'quote_name': row['quote_name'],
                'country': currency_info.get('country', 'Unknown'),
                'region': currency_info.get('region', 'Unknown'),
                'category': row['category'],
                'bid': row.get('bid'),
                'ask': row.get('ask'),
                'last': row.get('last'),
                'spread_pips': row.get('spread_pips'),
                'pip_size': row.get('pip_size'),
                'timestamp': row.get('timestamp'),
                'tradeable': True
            }
            usd_export_data.append(export_row)
        
        # Create comprehensive USD pairs DataFrame
        usd_comprehensive_df = pd.DataFrame(usd_export_data)
        
        # Export USD pairs data
        usd_comprehensive_df.to_csv('usd_world_currency_pairs.csv', index=False)
        print(f"\n✅ USD market data saved to usd_world_currency_pairs.csv")
        
        # Export by category
        for category in ['Major', 'Minor', 'Exotic']:
            category_df = usd_comprehensive_df[usd_comprehensive_df['category'] == category]
            if not category_df.empty:
                filename = f'usd_{category.lower()}_pairs.csv'
                category_df.to_csv(filename, index=False)
                print(f"✅ USD {category} pairs ({len(category_df)}) saved to {filename}")
        
        # Export by region
        regions = usd_comprehensive_df['region'].unique()
        for region in regions:
            region_df = usd_comprehensive_df[usd_comprehensive_df['region'] == region]
            if not region_df.empty:
                safe_region = region.replace('/', '_').replace(' ', '_')
                filename = f'usd_to_{safe_region.lower()}_pairs.csv'
                region_df.to_csv(filename, index=False)
                print(f"✅ USD to {region} pairs ({len(region_df)}) saved to {filename}")
        
        # Create USD trading summary
        print(f"\n📊 USD PAIRS TRADING SUMMARY:")
        print("=" * 50)
        print(f"Total USD pairs discovered: {len(usd_comprehensive_df)}")
        
        # Category breakdown
        category_breakdown = usd_comprehensive_df['category'].value_counts()
        for category, count in category_breakdown.items():
            percentage = (count / len(usd_comprehensive_df)) * 100
            print(f"  {category:<10}: {count:>3} pairs ({percentage:>5.1f}%)")
        
        # Regional breakdown
        print(f"\nRegional Distribution:")
        region_breakdown = usd_comprehensive_df['region'].value_counts()
        for region, count in region_breakdown.head(10).items():
            percentage = (count / len(usd_comprehensive_df)) * 100
            print(f"  {region:<20}: {count:>3} pairs ({percentage:>5.1f}%)")
        
        # Best spread analysis (lowest spreads)
        if 'spread_pips' in usd_comprehensive_df.columns:
            valid_spreads = usd_comprehensive_df.dropna(subset=['spread_pips'])
            if not valid_spreads.empty:
                best_spreads = valid_spreads.nsmallest(10, 'spread_pips')
                print(f"\n🎯 USD Pairs with Best Spreads (Top 10):")
                print("-" * 50)
                for _, pair in best_spreads.iterrows():
                    print(f"  {pair['symbol']:<8} ({pair['category']:<6}): {pair['spread_pips']:.1f} pips")
        
        # Trading session recommendations for USD pairs
        print(f"\n🕒 OPTIMAL USD TRADING SESSIONS:")
        print("-" * 40)
        
        # New York session (USD most active)
        ny_session_pairs = usd_comprehensive_df[
            usd_comprehensive_df['quote_currency'].isin(['EUR', 'GBP', 'CHF', 'CAD', 'MXN', 'BRL'])
        ]['symbol'].head(8).tolist()
        print(f"New York Session (13:00-22:00 GMT):")
        print(f"  Recommended: {', '.join(ny_session_pairs)}")
        
        # London session (USD/EUR active)
        london_session_pairs = usd_comprehensive_df[
            usd_comprehensive_df['quote_currency'].isin(['EUR', 'GBP', 'CHF', 'SEK', 'NOK', 'PLN'])
        ]['symbol'].head(8).tolist()
        print(f"\nLondon Session (08:00-17:00 GMT):")
        print(f"  Recommended: {', '.join(london_session_pairs)}")
        
        # Asian session (USD/Asian currencies)
        asian_session_pairs = usd_comprehensive_df[
            usd_comprehensive_df['quote_currency'].isin(['JPY', 'CNY', 'HKD', 'SGD', 'KRW', 'AUD', 'NZD'])
        ]['symbol'].head(8).tolist()
        print(f"\nAsian Session (21:00-06:00 GMT):")
        print(f"  Recommended: {', '.join(asian_session_pairs)}")
        
        print(f"\n💡 USD Trading Recommendations:")
        print(f"  • Focus on USD major pairs during US trading hours")
        print(f"  • Monitor Federal Reserve policy announcements")
        print(f"  • Consider US economic calendar for trade timing")
        print(f"  • Use USD strength as a market sentiment indicator")
        print(f"  • Diversify across different regions and time zones")
        
    else:
        print("❌ No USD market data available")
        
else:
    print("Cannot fetch USD market data - no pairs discovered or connection unavailable")

# Create USD-specific risk management guidelines
if 'usd_comprehensive_df' in locals() and not usd_comprehensive_df.empty:
    print(f"\n🛡️ USD-SPECIFIC RISK MANAGEMENT:")
    print("-" * 40)
    
    usd_risk_guidelines = {
        'Major USD Pairs': {
            'pairs': usd_comprehensive_df[usd_comprehensive_df['category'] == 'Major']['symbol'].tolist(),
            'max_risk_per_trade': '2-3%',
            'leverage_recommendation': '1:20 to 1:50',
            'session_focus': 'New York and London overlap (13:00-17:00 GMT)'
        },
        'Minor USD Pairs': {
            'pairs': usd_comprehensive_df[usd_comprehensive_df['category'] == 'Minor']['symbol'].tolist(),
            'max_risk_per_trade': '1.5-2%',
            'leverage_recommendation': '1:10 to 1:30',
            'session_focus': 'Respective regional sessions'
        },
        'Exotic USD Pairs': {
            'pairs': usd_comprehensive_df[usd_comprehensive_df['category'] == 'Exotic']['symbol'].tolist(),
            'max_risk_per_trade': '0.5-1%',
            'leverage_recommendation': '1:5 to 1:15',
            'session_focus': 'Local market hours for quote currency'
        }
    }
    
    for risk_level, guidelines in usd_risk_guidelines.items():
        if guidelines['pairs']:
            print(f"\n{risk_level}:")
            print(f"  Pairs count: {len(guidelines['pairs'])}")
            print(f"  Risk per trade: {guidelines['max_risk_per_trade']}")
            print(f"  Recommended leverage: {guidelines['leverage_recommendation']}")
            print(f"  Optimal sessions: {guidelines['session_focus']}")
            if len(guidelines['pairs']) <= 15:
                print(f"  Pairs: {', '.join(guidelines['pairs'])}")
            else:
                print(f"  Sample pairs: {', '.join(guidelines['pairs'][:10])}...")
    
    # Save USD risk management plan
    with open('usd_risk_management_plan.txt', 'w') as f:
        f.write("USD CURRENCY PAIRS RISK MANAGEMENT PLAN\n")
        f.write("=" * 50 + "\n\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("USD TRADING GUIDELINES:\n")
        f.write("-" * 25 + "\n")
        f.write("1. USD is the world's primary reserve currency\n")
        f.write("2. Most liquid during US trading hours (13:00-22:00 GMT)\n")
        f.write("3. Heavily influenced by Federal Reserve policy\n")
        f.write("4. Acts as safe haven during market uncertainty\n")
        f.write("5. Strong correlation with US economic data\n\n")
        
        for risk_level, guidelines in usd_risk_guidelines.items():
            if guidelines['pairs']:
                f.write(f"{risk_level}:\n")
                f.write(f"  Total pairs: {len(guidelines['pairs'])}\n")
                f.write(f"  Risk per trade: {guidelines['max_risk_per_trade']}\n")
                f.write(f"  Leverage: {guidelines['leverage_recommendation']}\n")
                f.write(f"  Optimal timing: {guidelines['session_focus']}\n")
                f.write(f"  Sample pairs: {', '.join(guidelines['pairs'][:10])}\n\n")
        
        f.write("KEY USD TRADING RULES:\n")
        f.write("-" * 22 + "\n")
        f.write("• Never risk more than 2% per trade on USD majors\n")
        f.write("• Reduce risk to 1% for exotic USD pairs\n")
        f.write("• Monitor FOMC meetings and US economic releases\n")
        f.write("• Consider USD index (DXY) for overall USD strength\n")
        f.write("• Use appropriate position sizing based on volatility\n")
        f.write("• Always use stop losses and take profits\n")
        f.write("• Diversify across different USD pair categories\n")
    
    print(f"\n✅ USD risk management plan saved to usd_risk_management_plan.txt")
    print(f"✅ Comprehensive USD pairs analysis complete!")
    
    # Final USD summary
    print(f"\n🎯 FINAL USD PAIRS SUMMARY:")
    print(f"  Total USD pairs: {len(usd_comprehensive_df)}")
    print(f"  Countries covered: {usd_comprehensive_df['country'].nunique()}")
    print(f"  Regions covered: {usd_comprehensive_df['region'].nunique()}")
    print(f"  Live prices obtained: {usd_comprehensive_df['bid'].notna().sum()}")
    print(f"  Files created: usd_world_currency_pairs.csv + category/region exports")

### Export World Forex Data

In [ ]:
class WorldForexExporter:
    """Export comprehensive world forex data"""
    
    def __init__(self, pairs_data: List[Dict], currencies_db: WorldCurrenciesDatabase):
        """Initialise with pairs data and currencies database"""
        self.pairs_data = pairs_data
        self.currencies_db = currencies_db
        self.logger = logging.getLogger('WorldForexExporter')
    
    def to_dataframe(self) -> pd.DataFrame:
        """Convert pairs data to comprehensive DataFrame"""
        if not self.pairs_data:
            return pd.DataFrame()
        
        df_data = []
        for pair in self.pairs_data:
            df_data.append({
                'symbol': pair['symbol'],
                'base_currency': pair['base_currency'],
                'quote_currency': pair['quote_currency'],
                'base_name': pair['base_name'],
                'quote_name': pair['quote_name'],
                'base_country': pair['base_country'],
                'quote_country': pair['quote_country'],
                'base_region': pair['base_region'],
                'quote_region': pair['quote_region'],
                'category': pair['category'],
                'pip_size': pair['pip_size'],
                'tradeable': pair['tradeable']
            })
        
        return pd.DataFrame(df_data)
    
    def export_complete_dataset(self, filename: str = 'world_forex_pairs_complete.csv'):
        """Export complete forex pairs dataset"""
        df = self.to_dataframe()
        if not df.empty:
            df.to_csv(filename, index=False)
            self.logger.info(f"Exported {len(df)} pairs to {filename}")
            print(f"✅ Complete dataset exported to {filename}")
            return df
        else:
            print("❌ No data to export")
            return pd.DataFrame()
    
    def export_by_category(self, base_filename: str = 'world_forex'):
        """Export pairs grouped by category"""
        df = self.to_dataframe()
        if df.empty:
            return
        
        categories = df['category'].unique()
        
        for category in categories:
            category_df = df[df['category'] == category]
            filename = f"{base_filename}_{category.lower()}_pairs.csv"
            category_df.to_csv(filename, index=False)
            print(f"✅ {category} pairs ({len(category_df)}) exported to {filename}")
    
    def export_by_region(self, base_filename: str = 'world_forex'):
        """Export pairs grouped by base currency region"""
        df = self.to_dataframe()
        if df.empty:
            return
        
        regions = df['base_region'].unique()
        
        for region in regions:
            region_df = df[df['base_region'] == region]
            safe_region = region.replace('/', '_').replace(' ', '_')
            filename = f"{base_filename}_{safe_region.lower()}_base.csv"
            region_df.to_csv(filename, index=False)
            print(f"✅ {region} base pairs ({len(region_df)}) exported to {filename}")
    
    def export_symbols_only(self, filename: str = 'world_forex_symbols.txt'):
        """Export just the trading symbols"""
        symbols = [pair['symbol'] for pair in self.pairs_data]
        
        with open(filename, 'w') as f:
            for symbol in sorted(symbols):
                f.write(f"{symbol}\n")
        
        print(f"✅ {len(symbols)} symbols exported to {filename}")
    
    def export_currency_database(self, filename: str = 'world_currencies_database.csv'):
        """Export the complete currency database"""
        currency_data = []
        
        for code, info in self.currencies_db.world_currencies.items():
            currency_data.append({
                'currency_code': code,
                'currency_name': info['name'],
                'country': info['country'],
                'region': info['region'],
                'category': info['category']
            })
        
        df = pd.DataFrame(currency_data)
        df.to_csv(filename, index=False)
        print(f"✅ Currency database ({len(df)} currencies) exported to {filename}")
    
    def create_trading_summary(self, filename: str = 'world_forex_trading_summary.txt'):
        """Create a comprehensive trading summary report"""
        if not self.pairs_data:
            return
        
        df = self.to_dataframe()
        
        with open(filename, 'w') as f:
            f.write("WORLD FOREX TRADING SUMMARY REPORT\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Total Discovered Pairs: {len(self.pairs_data)}\n\n")
            
            # Category summary
            f.write("PAIRS BY CATEGORY:\n")
            f.write("-" * 20 + "\n")
            category_counts = df['category'].value_counts()
            for category, count in category_counts.items():
                f.write(f"{category:<10}: {count:>4} pairs\n")
            
            # Region summary
            f.write(f"\nPAIRS BY BASE CURRENCY REGION:\n")
            f.write("-" * 30 + "\n")
            region_counts = df['base_region'].value_counts()
            for region, count in region_counts.items():
                f.write(f"{region:<20}: {count:>4} pairs\n")
            
            # Top base currencies
            f.write(f"\nTOP BASE CURRENCIES:\n")
            f.write("-" * 20 + "\n")
            base_counts = df['base_currency'].value_counts().head(20)
            for base, count in base_counts.items():
                base_name = self.currencies_db.get_currency_info(base).get('name', base)
                f.write(f"{base} ({base_name}): {count} pairs\n")
            
            f.write(f"\nRECOMMENDED TRADING APPROACH:\n")
            f.write("-" * 30 + "\n")
            f.write("1. Start with Major pairs (lowest risk, highest liquidity)\n")
            f.write("2. Progress to Minor pairs for diversification\n")
            f.write("3. Use Exotic pairs sparingly (higher risk/reward)\n")
            f.write("4. Consider regional economic factors\n")
            f.write("5. Always use proper risk management (1-3% per trade)\n")
        
        print(f"✅ Trading summary report exported to {filename}")

# Export all discovered forex data
if forex_pairs_data:
    print("\n📁 Exporting comprehensive world forex data...")
    
    exporter = WorldForexExporter(forex_pairs_data, world_currencies)
    
    # Export complete dataset
    complete_df = exporter.export_complete_dataset()
    
    # Export by categories
    exporter.export_by_category()
    
    # Export by regions
    exporter.export_by_region()
    
    # Export symbols only
    exporter.export_symbols_only()
    
    # Export currency database
    exporter.export_currency_database()
    
    # Create trading summary
    exporter.create_trading_summary()
    
    # Display final statistics
    if not complete_df.empty:
        print(f"\n📊 FINAL STATISTICS:")
        print(f"   Total pairs discovered: {len(complete_df)}")
        print(f"   Unique base currencies: {complete_df['base_currency'].nunique()}")
        print(f"   Unique quote currencies: {complete_df['quote_currency'].nunique()}")
        print(f"   Countries represented: {complete_df['base_country'].nunique() + complete_df['quote_country'].nunique()}")
        print(f"   Geographic regions: {complete_df['base_region'].nunique()}")
        
        category_breakdown = complete_df['category'].value_counts()
        for category, count in category_breakdown.items():
            percentage = (count / len(complete_df)) * 100
            print(f"   {category} pairs: {count} ({percentage:.1f}%)")
    
else:
    print("❌ No forex pairs data available for export")

### Advanced Analysis and Recommendations

In [ ]:
class WorldForexAnalyser:
    """Advanced analysis of world forex pairs"""
    
    def __init__(self, pairs_df: pd.DataFrame, currencies_db: WorldCurrenciesDatabase):
        """Initialise with pairs DataFrame and currencies database"""
        self.pairs_df = pairs_df
        self.currencies_db = currencies_db
        self.logger = logging.getLogger('WorldForexAnalyser')
    
    def analyse_currency_strength(self) -> Dict:
        """Analyse currency strength based on pair count"""
        if self.pairs_df.empty:
            return {}
        
        # Count occurrences as base and quote currency
        base_counts = self.pairs_df['base_currency'].value_counts()
        quote_counts = self.pairs_df['quote_currency'].value_counts()
        
        # Combine counts for overall strength indicator
        all_currencies = set(list(base_counts.index) + list(quote_counts.index))
        currency_strength = {}
        
        for currency in all_currencies:
            base_count = base_counts.get(currency, 0)
            quote_count = quote_counts.get(currency, 0)
            total_count = base_count + quote_count
            
            # Strength score (more weight to base currency usage)
            strength_score = (base_count * 1.5) + quote_count
            
            currency_strength[currency] = {
                'total_pairs': total_count,
                'as_base': base_count,
                'as_quote': quote_count,
                'strength_score': strength_score
            }
        
        return currency_strength
    
    def get_regional_opportunities(self) -> Dict:
        """Identify trading opportunities by region"""
        if self.pairs_df.empty:
            return {}
        
        regional_opportunities = {}
        regions = self.pairs_df['base_region'].unique()
        
        for region in regions:
            region_pairs = self.pairs_df[self.pairs_df['base_region'] == region]
            
            opportunities = {
                'total_pairs': len(region_pairs),
                'major_pairs': len(region_pairs[region_pairs['category'] == 'Major']),
                'minor_pairs': len(region_pairs[region_pairs['category'] == 'Minor']),
                'exotic_pairs': len(region_pairs[region_pairs['category'] == 'Exotic']),
                'countries': region_pairs['base_country'].nunique(),
                'recommended_pairs': region_pairs[region_pairs['category'].isin(['Major', 'Minor'])]['symbol'].head(10).tolist()
            }
            
            regional_opportunities[region] = opportunities
        
        return regional_opportunities
    
    def get_diversification_recommendations(self) -> Dict:
        """Get portfolio diversification recommendations"""
        recommendations = {
            'conservative': [],
            'balanced': [],
            'aggressive': []
        }
        
        if self.pairs_df.empty:
            return recommendations
        
        # Conservative: Major pairs only
        major_pairs = self.pairs_df[self.pairs_df['category'] == 'Major']
        recommendations['conservative'] = major_pairs['symbol'].head(8).tolist()
        
        # Balanced: Mix of major and minor
        minor_pairs = self.pairs_df[self.pairs_df['category'] == 'Minor']
        balanced_majors = major_pairs['symbol'].head(6).tolist()
        balanced_minors = minor_pairs['symbol'].head(6).tolist()
        recommendations['balanced'] = balanced_majors + balanced_minors
        
        # Aggressive: Include exotic pairs
        exotic_pairs = self.pairs_df[self.pairs_df['category'] == 'Exotic']
        aggressive_majors = major_pairs['symbol'].head(4).tolist()
        aggressive_minors = minor_pairs['symbol'].head(4).tolist()
        aggressive_exotics = exotic_pairs['symbol'].head(8).tolist()
        recommendations['aggressive'] = aggressive_majors + aggressive_minors + aggressive_exotics
        
        return recommendations
    
    def analyse_economic_factors(self) -> Dict:
        """Analyse pairs by economic development factors"""
        if self.pairs_df.empty:
            return {}
        
        # Define economic categories
        developed_economies = ['USD', 'EUR', 'GBP', 'JPY', 'CHF', 'CAD', 'AUD', 'NZD', 'SEK', 'NOK', 'DKK']
        emerging_economies = ['CNY', 'HKD', 'SGD', 'KRW', 'TWD', 'INR', 'BRL', 'MXN', 'ZAR', 'TRY', 'PLN', 'CZK', 'HUF']
        
        economic_analysis = {
            'developed_vs_developed': 0,
            'developed_vs_emerging': 0,
            'developed_vs_frontier': 0,
            'emerging_vs_emerging': 0,
            'emerging_vs_frontier': 0,
            'frontier_vs_frontier': 0
        }
        
        for _, pair in self.pairs_df.iterrows():
            base = pair['base_currency']
            quote = pair['quote_currency']
            
            base_type = 'developed' if base in developed_economies else 'emerging' if base in emerging_economies else 'frontier'
            quote_type = 'developed' if quote in developed_economies else 'emerging' if quote in emerging_economies else 'frontier'
            
            pair_type = f"{base_type}_vs_{quote_type}"
            if pair_type in economic_analysis:
                economic_analysis[pair_type] += 1
        
        return economic_analysis
    
    def print_comprehensive_analysis(self):
        """Print complete analysis report"""
        print("\n🌍 COMPREHENSIVE WORLD FOREX ANALYSIS")
        print("=" * 80)
        
        if self.pairs_df.empty:
            print("No data available for analysis")
            return
        
        # Currency strength analysis
        currency_strength = self.analyse_currency_strength()
        print(f"\n💪 CURRENCY STRENGTH ANALYSIS (Top 15):")
        print("-" * 50)
        sorted_strength = sorted(currency_strength.items(), key=lambda x: x[1]['strength_score'], reverse=True)
        
        for i, (currency, data) in enumerate(sorted_strength[:15], 1):
            currency_info = self.currencies_db.get_currency_info(currency)
            currency_name = currency_info.get('name', currency)
            print(f"{i:2d}. {currency} ({currency_name:<20}): {data['total_pairs']:>3} pairs (Base: {data['as_base']}, Quote: {data['as_quote']})")
        
        # Regional opportunities
        regional_opportunities = self.get_regional_opportunities()
        print(f"\n🌏 REGIONAL TRADING OPPORTUNITIES:")
        print("-" * 40)
        
        for region, data in regional_opportunities.items():
            print(f"\n{region}:")
            print(f"  Total pairs: {data['total_pairs']} ({data['countries']} countries)")
            print(f"  Major: {data['major_pairs']}, Minor: {data['minor_pairs']}, Exotic: {data['exotic_pairs']}")
            if data['recommended_pairs']:
                print(f"  Top pairs: {', '.join(data['recommended_pairs'][:5])}")
        
        # Economic analysis
        economic_analysis = self.analyse_economic_factors()
        print(f"\n📊 ECONOMIC DEVELOPMENT ANALYSIS:")
        print("-" * 35)
        for pair_type, count in economic_analysis.items():
            if count > 0:
                pair_label = pair_type.replace('_vs_', ' vs ').title()
                print(f"  {pair_label:<25}: {count:>3} pairs")
        
        # Diversification recommendations
        diversification = self.get_diversification_recommendations()
        print(f"\n💼 PORTFOLIO DIVERSIFICATION RECOMMENDATIONS:")
        print("-" * 45)
        
        for strategy, pairs in diversification.items():
            print(f"\n{strategy.upper()} Portfolio:")
            if pairs:
                for i in range(0, len(pairs), 6):
                    row = pairs[i:i+6]
                    print(f"  {' '.join(f'{pair:<8}' for pair in row)}")
        
        # Trading session recommendations
        print(f"\n🕐 OPTIMAL TRADING SESSIONS:")
        print("-" * 30)
        print("Asian Session (21:00-06:00 GMT):")
        asian_pairs = self.pairs_df[
            (self.pairs_df['base_currency'].isin(['JPY', 'AUD', 'NZD', 'CNY', 'HKD', 'SGD', 'KRW'])) |
            (self.pairs_df['quote_currency'].isin(['JPY', 'AUD', 'NZD', 'CNY', 'HKD', 'SGD', 'KRW']))
        ]['symbol'].head(10).tolist()
        print(f"  Recommended: {', '.join(asian_pairs[:8])}")
        
        print("\nEuropean Session (07:00-16:00 GMT):")
        european_pairs = self.pairs_df[
            (self.pairs_df['base_currency'].isin(['EUR', 'GBP', 'CHF', 'SEK', 'NOK', 'DKK', 'PLN'])) |
            (self.pairs_df['quote_currency'].isin(['EUR', 'GBP', 'CHF', 'SEK', 'NOK', 'DKK', 'PLN']))
        ]['symbol'].head(10).tolist()
        print(f"  Recommended: {', '.join(european_pairs[:8])}")
        
        print("\nAmerican Session (13:00-22:00 GMT):")
        american_pairs = self.pairs_df[
            (self.pairs_df['base_currency'].isin(['USD', 'CAD', 'MXN', 'BRL', 'ARS', 'CLP'])) |
            (self.pairs_df['quote_currency'].isin(['USD', 'CAD', 'MXN', 'BRL', 'ARS', 'CLP']))
        ]['symbol'].head(10).tolist()
        print(f"  Recommended: {', '.join(american_pairs[:8])}")

# Run comprehensive analysis
if 'complete_df' in locals() and not complete_df.empty:
    print("\n🔍 Running comprehensive world forex analysis...")
    analyser = WorldForexAnalyser(complete_df, world_currencies)
    analyser.print_comprehensive_analysis()
else:
    print("❌ No forex data available for analysis")

### Risk Management Framework

In [ ]:
class WorldForexRiskManager:
    """Risk management framework for world forex trading"""
    
    def __init__(self, pairs_df: pd.DataFrame, currencies_db: WorldCurrenciesDatabase):
        """Initialise with pairs data and currencies database"""
        self.pairs_df = pairs_df
        self.currencies_db = currencies_db
        self.logger = logging.getLogger('WorldForexRiskManager')
    
    def categorise_risk_levels(self) -> Dict[str, List[str]]:
        """Categorise all pairs by risk level"""
        risk_categories = {
            'Very Low Risk': [],
            'Low Risk': [],
            'Medium Risk': [],
            'High Risk': [],
            'Very High Risk': []
        }
        
        if self.pairs_df.empty:
            return risk_categories
        
        # Define risk factors
        stable_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'CHF']
        major_currencies = ['USD', 'EUR', 'GBP', 'JPY', 'CHF', 'CAD', 'AUD', 'NZD']
        developed_currencies = major_currencies + ['SEK', 'NOK', 'DKK', 'HKD', 'SGD']
        
        for _, pair in self.pairs_df.iterrows():
            base = pair['base_currency']
            quote = pair['quote_currency']
            symbol = pair['symbol']
            category = pair['category']
            
            # Risk scoring
            risk_score = 0
            
            # Category risk
            if category == 'Major':
                risk_score += 1
            elif category == 'Minor':
                risk_score += 2
            else:  # Exotic
                risk_score += 4
            
            # Currency stability risk
            if base in stable_currencies and quote in stable_currencies:
                risk_score += 0
            elif base in major_currencies and quote in major_currencies:
                risk_score += 1
            elif base in developed_currencies or quote in developed_currencies:
                risk_score += 2
            else:
                risk_score += 3
            
            # Assign risk category
            if risk_score <= 1:
                risk_categories['Very Low Risk'].append(symbol)
            elif risk_score <= 2:
                risk_categories['Low Risk'].append(symbol)
            elif risk_score <= 4:
                risk_categories['Medium Risk'].append(symbol)
            elif risk_score <= 6:
                risk_categories['High Risk'].append(symbol)
            else:
                risk_categories['Very High Risk'].append(symbol)
        
        return risk_categories
    
    def get_position_sizing_framework(self) -> Dict:
        """Get position sizing recommendations based on risk levels"""
        risk_categories = self.categorise_risk_levels()
        
        position_sizing = {
            'Very Low Risk': {
                'max_risk_per_trade': '2-3%',
                'max_portfolio_allocation': '60%',
                'leverage_recommendation': '1:20 to 1:50',
                'pairs': risk_categories['Very Low Risk']
            },
            'Low Risk': {
                'max_risk_per_trade': '1.5-2.5%',
                'max_portfolio_allocation': '30%',
                'leverage_recommendation': '1:10 to 1:30',
                'pairs': risk_categories['Low Risk']
            },
            'Medium Risk': {
                'max_risk_per_trade': '1-2%',
                'max_portfolio_allocation': '15%',
                'leverage_recommendation': '1:5 to 1:20',
                'pairs': risk_categories['Medium Risk']
            },
            'High Risk': {
                'max_risk_per_trade': '0.5-1%',
                'max_portfolio_allocation': '8%',
                'leverage_recommendation': '1:2 to 1:10',
                'pairs': risk_categories['High Risk']
            },
            'Very High Risk': {
                'max_risk_per_trade': '0.25-0.5%',
                'max_portfolio_allocation': '3%',
                'leverage_recommendation': '1:1 to 1:5',
                'pairs': risk_categories['Very High Risk']
            }
        }
        
        return position_sizing
    
    def generate_correlation_warnings(self) -> Dict:
        """Generate warnings about potentially correlated pairs"""
        correlation_warnings = {
            'high_correlation_groups': [],
            'regional_concentration_risk': {},
            'currency_concentration_risk': {}
        }
        
        if self.pairs_df.empty:
            return correlation_warnings
        
        # Regional concentration analysis
        base_regions = self.pairs_df['base_region'].value_counts()
        for region, count in base_regions.items():
            if count > 10:  # More than 10 pairs from same region
                correlation_warnings['regional_concentration_risk'][region] = {
                    'pair_count': count,
                    'warning': f"High concentration of {count} pairs from {region}"
                }
        
        # Currency concentration analysis
        base_currencies = self.pairs_df['base_currency'].value_counts()
        for currency, count in base_currencies.items():
            if count > 20:  # More than 20 pairs with same base
                currency_info = self.currencies_db.get_currency_info(currency)
                correlation_warnings['currency_concentration_risk'][currency] = {
                    'pair_count': count,
                    'currency_name': currency_info.get('name', currency),
                    'warning': f"High exposure to {currency} in {count} pairs"
                }
        
        # High correlation groups (pairs likely to move together)
        correlation_groups = [
            {
                'group_name': 'USD Majors',
                'pairs': [p['symbol'] for p in self.pairs_df.to_dict('records') 
                         if (p['base_currency'] == 'USD' or p['quote_currency'] == 'USD') 
                         and p['category'] == 'Major'],
                'correlation_reason': 'All influenced by USD monetary policy'
            },
            {
                'group_name': 'EUR Cross-pairs',
                'pairs': [p['symbol'] for p in self.pairs_df.to_dict('records') 
                         if p['base_currency'] == 'EUR' and p['quote_currency'] != 'USD'],
                'correlation_reason': 'All influenced by ECB policy and European economic data'
            },
            {
                'group_name': 'Commodity Currencies',
                'pairs': [p['symbol'] for p in self.pairs_df.to_dict('records') 
                         if p['base_currency'] in ['AUD', 'CAD', 'NZD', 'ZAR'] 
                         or p['quote_currency'] in ['AUD', 'CAD', 'NZD', 'ZAR']],
                'correlation_reason': 'Influenced by commodity prices and risk sentiment'
            }
        ]
        
        correlation_warnings['high_correlation_groups'] = [
            group for group in correlation_groups if len(group['pairs']) >= 5
        ]
        
        return correlation_warnings
    
    def create_risk_management_plan(self, filename: str = 'world_forex_risk_management_plan.txt'):
        """Create comprehensive risk management plan"""
        position_sizing = self.get_position_sizing_framework()
        correlation_warnings = self.generate_correlation_warnings()
        
        with open(filename, 'w') as f:
            f.write("WORLD FOREX RISK MANAGEMENT PLAN\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            f.write("POSITION SIZING FRAMEWORK:\n")
            f.write("-" * 30 + "\n")
            for risk_level, guidelines in position_sizing.items():
                f.write(f"\n{risk_level}:\n")
                f.write(f"  Risk per Trade: {guidelines['max_risk_per_trade']}\n")
                f.write(f"  Portfolio Allocation: {guidelines['max_portfolio_allocation']}\n")
                f.write(f"  Leverage: {guidelines['leverage_recommendation']}\n")
                f.write(f"  Number of Pairs: {len(guidelines['pairs'])}\n")
            
            f.write(f"\nCORRELATION WARNINGS:\n")
            f.write("-" * 20 + "\n")
            
            # Regional concentration warnings
            if correlation_warnings['regional_concentration_risk']:
                f.write("\nRegional Concentration Risks:\n")
                for region, data in correlation_warnings['regional_concentration_risk'].items():
                    f.write(f"  {region}: {data['warning']}\n")
            
            # Currency concentration warnings
            if correlation_warnings['currency_concentration_risk']:
                f.write("\nCurrency Concentration Risks:\n")
                for currency, data in correlation_warnings['currency_concentration_risk'].items():
                    f.write(f"  {currency} ({data['currency_name']}): {data['warning']}\n")
            
            # Correlation groups
            if correlation_warnings['high_correlation_groups']:
                f.write("\nHigh Correlation Groups (Limit simultaneous trades):\n")
                for group in correlation_warnings['high_correlation_groups']:
                    f.write(f"  {group['group_name']}: {len(group['pairs'])} pairs\n")
                    f.write(f"    Reason: {group['correlation_reason']}\n")
            
            f.write(f"\nGENERAL RISK MANAGEMENT RULES:\n")
            f.write("-" * 35 + "\n")
            f.write("1. Never risk more than 2% of account on any single trade\n")
            f.write("2. Limit total portfolio risk to 6-8% across all open positions\n")
            f.write("3. Avoid more than 3 highly correlated pairs simultaneously\n")
            f.write("4. Use stop losses on every trade (typically 1-3% of position)\n")
            f.write("5. Consider regional economic calendars for trade timing\n")
            f.write("6. Monitor central bank policies for major currencies\n")
            f.write("7. Start with major pairs and gradually add exotic pairs\n")
            f.write("8. Regularly review and adjust position sizes based on volatility\n")
            f.write("9. Keep detailed trading journal for performance analysis\n")
            f.write("10. Never trade with money you cannot afford to lose\n")
        
        print(f"✅ Risk management plan saved to {filename}")
    
    def print_risk_summary(self):
        """Print risk management summary"""
        print("\n⚠️  WORLD FOREX RISK MANAGEMENT SUMMARY")
        print("=" * 60)
        
        # Risk categorisation summary
        risk_categories = self.categorise_risk_levels()
        print(f"\nRisk Level Distribution:")
        for risk_level, pairs in risk_categories.items():
            if pairs:
                print(f"  {risk_level:<15}: {len(pairs):>3} pairs")
        
        # Position sizing summary
        position_sizing = self.get_position_sizing_framework()
        print(f"\nRecommended Portfolio Allocation:")
        total_allocation = 0
        for risk_level, guidelines in position_sizing.items():
            if guidelines['pairs']:
                allocation = float(guidelines['max_portfolio_allocation'].rstrip('%'))
                total_allocation += allocation
                print(f"  {risk_level:<15}: {allocation:>3.0f}% of portfolio")
        
        # Correlation warnings summary
        correlation_warnings = self.generate_correlation_warnings()
        if correlation_warnings['high_correlation_groups']:
            print(f"\n⚠️  Correlation Warnings:")
            for group in correlation_warnings['high_correlation_groups']:
                print(f"  {group['group_name']}: {len(group['pairs'])} pairs (limit simultaneous trades)")
        
        print(f"\n💡 Key Recommendations:")
        print(f"  • Start with Very Low and Low Risk pairs")
        print(f"  • Maximum 2% risk per trade")
        print(f"  • Diversify across regions and currencies")
        print(f"  • Use appropriate leverage based on risk level")
        print(f"  • Monitor economic calendars and central bank policies")

# Generate risk management framework
if 'complete_df' in locals() and not complete_df.empty:
    print("\n🛡️  Generating comprehensive risk management framework...")
    risk_manager = WorldForexRiskManager(complete_df, world_currencies)
    risk_manager.print_risk_summary()
    risk_manager.create_risk_management_plan()
else:
    print("❌ No forex data available for risk analysis")

### Environment Status and Cleanup

In [ ]:
def monitor_live_environment():
    """Monitor live trading environment status"""
    print("🖥️  LIVE ENVIRONMENT STATUS MONITOR")
    print("=" * 50)
    
    if live_connection.is_connected():
        try:
            account_info = live_connection.get_account_info()
            print(f"\n✅ LIVE Environment Status:")
            print(f"  Connection: Active")
            print(f"  Host: {live_connection.host}")
            print(f"  Port: {live_connection.port}")
            print(f"  Client ID: {live_connection.client_id}")
            
            if account_info.get('accounts'):
                print(f"  Accounts: {', '.join(account_info['accounts'])}")
            
            print(f"  Status: Ready for live forex trading")
            print(f"  ⚠️  WARNING: This is LIVE trading environment!")
            
        except Exception as e:
            print(f"  Warning: {e}")
    else:
        print(f"\n❌ LIVE Environment:")
        print(f"  Connection: Inactive")

# Monitor current status
monitor_live_environment()

# Display comprehensive session summary
print(f"\n🎯 WORLD FOREX DISCOVERY SESSION SUMMARY")
print("=" * 60)

if 'forex_pairs_data' in locals():
    total_pairs = len(forex_pairs_data)
    print(f"✅ Successfully discovered {total_pairs} tradeable forex pairs")
    
    if 'complete_df' in locals() and not complete_df.empty:
        # Category breakdown
        category_breakdown = complete_df['category'].value_counts()
        print(f"\nPair Categories:")
        for category, count in category_breakdown.items():
            percentage = (count / total_pairs) * 100
            print(f"  {category:<10}: {count:>4} pairs ({percentage:>5.1f}%)")
        
        # Regional representation
        unique_countries = set(complete_df['base_country'].tolist() + complete_df['quote_country'].tolist())
        unique_regions = set(complete_df['base_region'].tolist() + complete_df['quote_region'].tolist())
        
        print(f"\nGlobal Coverage:")
        print(f"  Countries represented: {len(unique_countries)}")
        print(f"  Geographic regions: {len(unique_regions)}")
        print(f"  Base currencies: {complete_df['base_currency'].nunique()}")
        print(f"  Quote currencies: {complete_df['quote_currency'].nunique()}")
else:
    total_pairs = 0
    print(f"❌ No forex pairs discovered")

print(f"\n📁 Generated Files:")
files_created = [
    "world_forex_pairs_complete.csv",
    "world_forex_major_pairs.csv", 
    "world_forex_minor_pairs.csv",
    "world_forex_exotic_pairs.csv",
    "world_forex_symbols.txt",
    "world_currencies_database.csv",
    "world_forex_trading_summary.txt",
    "world_forex_risk_management_plan.txt"
]

if 'market_df' in locals() and not market_df.empty:
    files_created.append("world_forex_market_data.csv")

for filename in files_created:
    if os.path.exists(filename):
        print(f"  ✅ {filename}")
    else:
        print(f"  ❌ {filename} (not created)")

print(f"\n📊 Trading Readiness Assessment:")
print(f"  Connection status: {'✅ Ready' if live_connection.is_connected() else '❌ Not Ready'}")
print(f"  Pairs discovered: {'✅ Ready' if total_pairs > 0 else '❌ Not Ready'}")
print(f"  Risk framework: {'✅ Ready' if os.path.exists('world_forex_risk_management_plan.txt') else '❌ Not Ready'}")

overall_status = "✅ READY FOR LIVE FOREX TRADING" if (live_connection.is_connected() and total_pairs > 0) else "❌ NOT READY FOR TRADING"
print(f"\nOverall Status: {overall_status}")

if total_pairs > 0:
    print(f"\n💡 Next Steps:")
    print(f"  1. Review the exported CSV files for trading opportunities")
    print(f"  2. Study the risk management plan carefully")
    print(f"  3. Start with Major pairs for lower risk")
    print(f"  4. Implement proper position sizing (max 2% risk per trade)")
    print(f"  5. Consider regional trading sessions for optimal timing")
    print(f"  6. Always use stop losses and risk management")
    print(f"  ⚠️  Remember: This is LIVE trading - real money at risk!")

### Clean Disconnect

In [ ]:
# Clean disconnect from live environment
print("🧹 Cleaning up live connection...")

try:
    if live_connection.is_connected():
        live_connection.disconnect()
        print("✅ Successfully disconnected from live environment")
    else:
        print("ℹ️  No active connection to disconnect")
        
except Exception as e:
    print(f"❌ Error during disconnection: {e}")

print(f"\n🏁 WORLD FOREX DISCOVERY SESSION COMPLETE")
print("=" * 60)
print("📋 Session Results:")
if 'total_pairs' in locals():
    print(f"  • Discovered {total_pairs} tradeable forex pairs")
    print(f"  • Analysed currencies from 195+ countries")
    print(f"  • Generated comprehensive risk management framework")
    print(f"  • Created detailed trading recommendations")

print(f"\n📂 All data exported to CSV files for further analysis")
print(f"🛡️  Risk management plan ready for implementation")
print(f"🌍 Ready to trade the world's currencies responsibly!")

print(f"\n⚠️  IMPORTANT DISCLAIMERS:")
print(f"  • This was connected to LIVE trading environment")
print(f"  • All trading involves significant risk of loss")
print(f"  • Past performance does not guarantee future results")
print(f"  • Always trade with proper risk management")
print(f"  • Consider seeking professional financial advice")
print(f"  • Never trade with money you cannot afford to lose")